In [1]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import random
import yaml
from pathlib import Path
from utility import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

income_categories_bins = params['income_categories_bins']
preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']

In [3]:
#create folders to save files
summary_dir = params['summary_dir']
preprocess_dir = _join(ctramp_dir, '_pre_process_files')

Path(summary_dir).mkdir(parents=True, exist_ok=True)
Path(preprocess_dir).mkdir(parents=True, exist_ok=True)

In [4]:
# outputs of CT-RAMP model for tour and trip file
household_model_dir = _join(model_outputs_dir, "main")

# input household and person data
person_file = _join(ctramp_dir, 'main\\personData_' + str(iteration) + '.csv')
household_file = _join(ctramp_dir, 'main\\householdData_' + str(iteration) + '.csv')

person = pd.read_csv(person_file)

hh = pd.read_csv(household_file, usecols = ['hh_id', 'taz'])
hh = hh.rename(columns = {'taz': 'home_zone'})

#taz to RDM zones, super districts, county
geo_cwks = pd.read_csv(_join(params['common_dir'], "geographies.csv")) #columns taz, rdm_zones, super_district, county

#taz to priority population
pp_perc = pd.read_excel(_join(params['common_dir'], "TAZ_Tract_cwk_summary.xlsx")) #columns = taz, pp_share 

# transbay od pairs
transbay_od = pd.read_csv(_join(params['common_dir'], "transbay_od.csv")) #columns = transbay_o, transbay_d

demand_matrices_dir = _join(model_outputs_dir, "demand_matrices")
transit_demand_dir = _join(demand_matrices_dir, "transit")
transit_skims_dir = _join(skims_dir, "transit")
highway_skims_dir = _join(skims_dir, "highway")

In [6]:
%%time
#create trips
df_trips = create_trip_roster(ctramp_dir, hh, pp_perc ,transbay_od, geo_cwks, link21_purp_mapping, iteration)

df_trips = df_trips.drop(columns=['person_num', 'tour_id', 'stop_id', 'orig_walk_segment',
                                  'tour_purpose', 'orig_purpose', 'dest_purpose',
                                 'dest_walk_segment', 'parking_taz', 'tour_mode', 'tour_category',
                                 'avAvailable', 'num_participants', 'new_dest_purp',
                                 'new_orig_purp', 'link21_tour_purp', 'link21_orig_purp',
                                 'link21_dest_purp', 'taxiWait', 'singleTNCWait', 'sharedTNCWait'])

df_trips['Period'] = df_trips['depart_hour'].map(time_period_mapping)
df_trips['Mode'] = df_trips['trip_mode'].map(mode_cat_mapping)

df_trips.to_parquet(_join(preprocess_dir, 'trip_roster.parquet'))

NAs in PP Share: 0


In [10]:
df_tours = create_tour_roster(ctramp_dir, hh, pp_perc ,transbay_od, geo_cwks, iteration)

df_tours = df_tours.drop(columns=['person_num', 'person_type', 'tour_id',
                                   'tour_category', 'orig_walk_segment', 'dest_walk_segment',
                                  'atWork_freq', 'num_ob_stops', 'num_ib_stops', 'avAvailable',
                                  'dcLogsum', 'origTaxiWait', 'destTaxiWait', 'origSingleTNCWait',
                                  'destSingleTNCWait', 'origSharedTNCWait','destSharedTNCWait', 
                                  'tour_composition', 'tour_participants'])

df_tours['Period'] = df_tours['start_hour'].map(time_period_mapping)
df_tours['Mode'] = df_tours['tour_mode'].map(mode_cat_mapping)

df_tours.to_parquet(_join(preprocess_dir, 'tour_roster.parquet'))